# **Анализ репозиториев в GitHub**

## 1. Введение
### 1.1 Цель исследования
Собрать данные о популярных GitHub-репозиториях по тегам AI и ML с помощью веб-парсинга результатов глобального поиска GitHub, провести очистку данных, создание новых признаков и исследовательский анализ, чтобы выявить закономерности популярности open-source проектов.

### 1.2 Источник данных

Мы выполняем парсинг HTML-страниц глобального поиска GitHub:

Глобальный поиск по тегу AI:
https://github.com/search?q=AI&type=repositories&s=stars&o=desc

Глобальный поиск по тегу ML:
https://github.com/search?q=ML&type=repositories&s=stars&o=desc

### 1.3 Описание предметной области

GitHub — крупнейшая платформа для open-source разработки.
Мы анализируем репозитории, отсортированные по популярности (метрика: количество звёзд, сортировка Most stars → по убыванию).
Это позволяет выявлять факторы, связанные с востребованностью проектов в области AI/ML.

---
### 1.4 Требования и результат

| Требование                             | Результат |
|:--------------------------------------:|:-----------------------------:|
| Количество наблюдений  > 1000          | 1685                         |
| Начальное количество признаков >= 8-10 | 10                            |
| Типы признаков                         | числовые (stars, forks, activity), категориальные (язык), даты (created/updated), бинарные (flags по стеку) |
| Количество новых признаков >= 2        | 13                           |
| Финальное количество признаков         | 23                            |
| Визуализации >= 3                      | 8                             |
| Итоговый формат                        | Jupyter/Colab notebook + .CSV |
| [дополнительно] Репозиторий на github  | ссылка                        |


### 1.5 Технологии проекта

| Технология          | Для чего используем                               | Документация                                       |
| :------------------ | :------------------------------------------------- | :------------------------------------------------- |
| Python 3.12.12      | основная среда выполнения                          | https://docs.python.org/3/                         |
| pandas              | работа с таблицами и предварительный анализ        | https://pandas.pydata.org/docs/                    |
| NumPy               | численные операции и векторные расчёты             | https://numpy.org/doc/                             |
| requests            | отправка HTTP-запросов к GitHub                    | https://requests.readthedocs.io/                   |
| BeautifulSoup (bs4) | парсинг HTML-страниц результатов поиска            | https://beautiful-soup-4.readthedocs.io/en/latest/ |
| seaborn             | быстрые статистические графики                     | https://seaborn.pydata.org/                        |
| matplotlib          | базовая визуализация и настройка осей              | https://matplotlib.org/stable/                     |
| ast                 | безопасное преобразование строк в списки/словари   | https://docs.python.org/3/library/ast.html         |
| time (stdlib)       | паузы между запросами                              | https://docs.python.org/3/library/time.html        |
| random (stdlib)     | рандомизация задержек при парсинге                 | https://docs.python.org/3/library/random.html      |
| itertools (stdlib)  | вспомогательные итераторы при обходе данных        | https://docs.python.org/3/library/itertools.html   |
| pathlib (stdlib)    | работа с путями и файлами                          | https://docs.python.org/3/library/pathlib.html     |


### 1.6 Команда и распределение задач

| Участник                         | Обязанности                         |
|----------------------------------|-------------------------------------|
| **Трофимов Матвей Владимирович** | Выбор первичных признаков, сбор и парсинг, первичные выводы     |
| **Самунджян Дина Арменаковна**   | Очистка данных, пропуски, дубликаты, нормализация типов |
| **Серенко Елена Валерьевна**     | Создание новых признаков, промежуточные выводы по сформированному датасету            |
| **Пащенко Дмитрий Игоревич**     | Визуализации, промежуточные выводы по анализу и результаты   |

### Структура ноутбука

```markdown
# 1. Введение
## 1.1 Цель проекта
## 1.2 Источник данных
## 1.3 Описание предметной области
## 1.4 Требования к датасету
## 1.5 Технологии проекта
## 1.6 Команда и распределение задач

Трофимов Матвей Владимирович
### 2. Сбор и парсинг данных
2.1 Загрузка библиотек и модулей для проекта
2.2 Функции для парсинга
2.3 Сбор репозиториев (AI и ML) и объединение
2.4 Набор полученных после парсинга признаков
2.5 Парсинг. Промежуточные выводы

Самунджян Дина Арменаковна
### 3. Очистка данных
3.1 Общий обзор датасета (shape, info, describe)
3.2 Обработка пропусков и заполнение описаний
3.3 Удаление дубликатов
3.4 Преобразование Programming languages и флага languages_missing
3.5 Очистка от выбросов по нескольким метрикам
3.6 Очистка. Промежуточные выводы

Серенко Елена Валерьевна
### 4. Создание новых признаков
4.1 Генерация временных меток Created и Last updated
4.2 Подробное описание исходных столбцов
4.3 Расчёт новых метрик вовлечённости и стека
4.4 Блок разведочного анализа по активности сообществ
4.5 Словарь новых признаков
4.6 Обогащение. Промежуточные выводы

Пащенко Дмитрий Игоревич
### 5. Анализ данных и визуализация
5.1 Описательная статистика
5.2 Популярность языков
5.3 Связи метрик и активность
5.4 Итоговые выводы EDA


### 6. Итоговые выводы всего исследования
```


# 2. Сбор и парсинг данных

In [ ]:
"""
Переключатель окружения для работы локально или в Google Colab.
Устанавливайте USE_COLAB = True при запуске ноутбука в Colab или False, если хотите запускать локально.
И в том и в другом случае вами нужно задать путь к рабочей директории! Спасибо:)
"""
from pathlib import Path

USE_COLAB = True # ТУТ ПЕРЕКЛЮЧАТЬ!

if USE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    DATA_DIR = Path("/content/drive/MyDrive/Colab Notebooks") # Тут указана наша папка по умолчанию (/content/drive/MyDrive/Colab Notebooks), если у вас другая, можете изменить путь
else:
    DATA_DIR = Path("укажите адрес вашей локальной папки, где лежат датасеты и куда будут они сохраняться")

print(f"Директория данных: {DATA_DIR.resolve()}")

# Далее в коде пути заменены на переменную DATA_DIR для удобства

## 2.1 Загрузка библиотек и модулей для проекта

In [ ]:
import sys
import requests
import time
import pandas as pd
import random
import bs4
from bs4 import BeautifulSoup
import itertools
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import ast

NoneType = type(None)

print("Python:", sys.version.split()[0])
print("requests:", requests.__version__)
print("pandas:", pd.__version__)
print("bs4:", bs4.__version__)
print("numpy:", np.__version__)
print("sns:", sns.__version__)
print("matplotlib:", matplotlib.__version__)

### 2.2 Функции для парсинга

**`repository_parsing(url)`**  
Функция принимает URL репозитория GitHub и с помощью HTTP‑запроса и библиотеки BeautifulSoup извлекает его параметры: название, описание, языки программирования, количество форков, звезд, коммитов, пул‑реквестов, контрибьюторов и релизов. При ошибке парсинга или отсутствии нужных элементов возвращает `None`, чтобы проблемные репозитории можно было безопасно пропустить при массовом сборе данных.


**`get_df_from_github(repositories_count)`**  
Функция выполняет массовый сбор данных о репозиториях по запросу `AI` и `ML` в поиске GitHub, обходя до 100 страниц выдачи с паузами между запросами. Для каждой найденной ссылки вызывается `repository_parsing`, а собранные параметры добавляются в списки до достижения заданного количества репозиториев `repositories_count`, после чего формируется `pandas.DataFrame` с основными метриками и ссылками на репозитории.


**`run_parsing()`**
Функция инициализации пользователя для упрощения работы с ноутбуком. Будьте внимательны и читайте, что она у вас спрашивает.

Смотрите подробности ниже в комментариях к коду.

In [ ]:
def run_parsing():
    response = input("""Хотите запустить парсинг данных?

    Введите 'yes' для запуска или 'no' для пропуска (мы учли любой регистр, не волнуйтесь):

    В случае YES -- это может занять около 2 часов.

    В случае NO -- у вас в директории должны лежать два файла github_repositories_AI.csv и github_repositories_ML.csv.

    Если что, скачать их можно вот тут: https://drive.google.com/drive/folders/1g35lFmY8D1aj2Smb53L9YXjkrm6WfI69

    Удачи :)

    """).strip().lower()

    if response == 'yes':
        print("Запускаем парсинг данных... Это займет некоторое время.")

        # Запрашиваем количество репозиториев для парсинга у нашего преподавателя или его ассистентов :)
        ai_repositories = input("Введите количество репозиториев для парсинга по тегу 'AI' (положительное целое число от 1 до 1000): ")
        while not ai_repositories.isdigit() or int(ai_repositories) <= 0 or int(ai_repositories) > 1000:
            ai_repositories = input("Пожалуйста, введите корректное количество репозиториев для AI (положительное целое число от 1 до 1000): ")

        ai_repositories = int(ai_repositories)

        ml_repositories = input("Введите количество репозиториев для парсинга по тегу 'ML'(положительное целое число от 1 до 1000): ")
        while not ml_repositories.isdigit() or int(ml_repositories) <= 0:
            ml_repositories = input("Пожалуйста, введите корректное количество репозиториев для ML (положительное целое число от 1 до 1000): ")

        ml_repositories = int(ml_repositories)

        df1 = get_df_from_github(ai_repositories, 'AI')
        df2 = get_df_from_github(ml_repositories, 'ML')

        return df1, df2

    elif response == 'no':
        print("Пропускаем парсинг. Загружаем датафреймы.")
        return None, None
    else:
        print("Некорректный ввод. Пожалуйста, введите 'yes' или 'no'.")
        return run_parsing()


In [ ]:
# Получаем на вход ссылку на репозиторий в GitHub и парсим содержимое
def repository_parsing(url):
    try:
        print(f'Парсим репозиторий: {url}')
        site1 = requests.get(url)
        site_text = site1.content.decode('utf-8')
        soup = BeautifulSoup(site_text, 'html.parser')
        split_url = url.split('/')
        repository_name = split_url[-2] + ' ' + split_url[-1]

        contributors = 0
        releases = 0

        # Считываем количество релизов со страницы репозитория.
        for a in soup.find_all('a', class_='Link--primary no-underline Link'):
            splitted = a.text.split()
            if splitted[0] == 'Releases' and len(splitted) > 1:
                releases = int(splitted[1].replace(',', '').replace("+", ""))

        # Считываем число контрибьюторов со страницы репозитория.
        for a in soup.find_all('a', class_='Link--primary no-underline Link d-flex flex-items-center'):
            splitted = a.text.split()
            if splitted[0] == 'Contributors' and len(splitted) > 1:
                contributors = int(splitted[1].replace(',', '').replace("+", ""))

        # Собираем перечень языков, указанных на странице репозитория.
        languages = []
        for span in soup.find_all('span', 'color-fg-default text-bold mr-1'):
            languages.append(span.text)

        # Инициализируем числовые метрики проекта.
        pull_requests = 0
        fork_amount = 0
        stars_amount = 0
        commits_amount = 0

        # Берём описание репозитория, если оно присутствует.
        if soup.find('p', class_='f4 my-3') != NoneType:
            repository_description = soup.find('p', class_='f4 my-3').text.strip()

        # Читаем счётчик открытых pull request.
        if soup.find('span', class_='Counter', id='pull-requests-repo-tab-count') != NoneType:
            pull_requests = int(soup.find('span', class_='Counter', id='pull-requests-repo-tab-count').get('title').replace(',', ''))

        # Читаем количество форков.
        if soup.find('span', id="repo-network-counter") != NoneType:
            fork_amount = int(soup.find('span', id="repo-network-counter").get('title').replace(',', ''))

        # Читаем количество звёзд.
        if soup.find('span', id="repo-stars-counter-star") != NoneType:
            stars_amount = int(soup.find('span', id="repo-stars-counter-star").get('title').replace(',', ''))

        # Читаем количество коммитов.
        if soup.find('span', class_='fgColor-default') != NoneType:
            commits_amount = int(soup.find('span', class_='fgColor-default').text.split()[0].replace(',', ''))
        # возвращаем список с параметрами репозитория
        return [repository_name, repository_description, languages, fork_amount, stars_amount, commits_amount, pull_requests, contributors, releases, url]

    except Exception as e:
        print(f"Ошибка при парсинге {url}: {e}")
        return None


In [ ]:
def get_df_from_github(repositories_count, query_tag):
    # Накопительные списки для формирования итогового DataFrame по репозиториям.
    names = []
    descriptions = []
    languages = []
    fork_amounts = []
    stars_amounts = []
    commits_amounts = []
    pull_requests_amounts = []
    contributors_amounts = []
    releases_amounts = []
    urls = []
    source_path = 'https://github.com'
    cur_repositories_count = 0
    site1 = requests.get(f'https://github.com/search?q={query_tag}&type=repositories&s=stars&o=desc')
    pages_count = repositories_count // 10 if repositories_count % 10 == 0 else repositories_count // 10 + 1
    i = 0
    while i < pages_count:
        i += 1
        if (i != 1):
            time.sleep(10)  # Ставим паузу между страницами, чтобы не спамить GitHub (встретили проблему блокировок при высокочастотном парсинге).
        print(f'Парсим репозитории с данного url: https://github.com/search?q={query_tag}&type=repositories&s=stars&o=desc&p={i}')
        site1 = requests.get(f'https://github.com/search?q={query_tag}&type=repositories&s=stars&o=desc&p={i}')
        print(f'Страница получена, с кодом ответа: {site1.status_code}')
        while (site1.status_code != 200):  # Повторяем запрос, пока не получим корректный ответ.
            time.sleep(10)
            site1 = requests.get(f'https://github.com/search?q={query_tag}&type=repositories&s=stars&o=desc&p={i}')
            print(f'Страница получена, с кодом ответа: {site1.status_code}')
        site_text = site1.content.decode('utf-8')
        soup = BeautifulSoup(site_text, 'html.parser')
        for link in soup.find_all('a', class_='Link__StyledLink-sc-1syctfj-0 prc-Link-Link-85e08'):
            if cur_repositories_count == repositories_count:  # Прекращаем сбор, как только набрали нужное число репозиториев.
                break
            repository_stat = repository_parsing(source_path + link.get('href'))
            if repository_stat == None:  # Пропускаем репозиторий, если парсинг завершился ошибкой.
                continue
            cur_repositories_count += 1
            # Сохраняем атрибуты репозитория для последующей сборки DataFrame.
            names.append(repository_stat[0])
            descriptions.append(repository_stat[1])
            languages.append(repository_stat[2])
            fork_amounts.append(repository_stat[3])
            stars_amounts.append(repository_stat[4])
            commits_amounts.append(repository_stat[5])
            pull_requests_amounts.append(repository_stat[6])
            contributors_amounts.append(repository_stat[7])
            releases_amounts.append(repository_stat[8])
            urls.append(repository_stat[9])
    df = pd.DataFrame({
        'Name': names,
        'Description': descriptions,
        'Programming languages': languages,
        'Forks count': fork_amounts,
        'Stars count': stars_amounts,
        'Commits count': commits_amounts,
        'Pull requests count': pull_requests_amounts,
        'Contributors count': contributors_amounts,
        'Releases counts': releases_amounts,
        'Url': urls
    })
    return df


In [ ]:
df1, df2 = run_parsing()

# Если парсинг был пропущен, можно загрузить данные из файлов.
if df1 is None or df2 is None:
    print("Загружаем датафреймы из CSV.")
    df1 = pd.read_csv(DATA_DIR / 'github_repositories_AI.csv')
    df2 = pd.read_csv(DATA_DIR / 'github_repositories_ML.csv')

In [ ]:
# df1 = get_df_from_github(50, 'AI') # передаем кол-во репозиториев, которое хотим спарсить и тег, по которому производим поиск
# df2 = get_df_from_github(50, 'ML') # передаем кол-во репозиториев, которое хотим спарсить и тег, по которому производим поиск

In [ ]:
df1.to_csv(DATA_DIR / 'github_repositories_AI.csv', index=False)
df2.to_csv(DATA_DIR / 'github_repositories_ML.csv', index=False)

### 2.3 Сбор репозиториев (AI и ML) и объединение

Совмещаем полученные датасеты в один.

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df.to_csv(DATA_DIR / 'github_repositories.csv', index=False)
df.shape

### Краткий словарь терминов
- Stars (звёзды) — отметки пользователей, показывающие интерес к проекту; используют как метрику популярности.
- Forks (форки) — копии репозитория, созданные для собственных изменений; отражают вовлечённость и желание доработать проект.
- Commits (коммиты) — сохранения изменений в репозитории; показывают историю развития.
- Pull requests (PR) — запросы на принятие изменений; число PR помогает оценить активность сообщества.
- Contributors (контрибьюторы) — уникальные участники, отправлявшие изменения; важны для оценки «живости» проекта.
- Releases (релизы) — официальные версии с зафиксированными изменениями; индикатор стабильности и зрелости продукта.

### 2.4 Набор полученных после парсинга признаков

- Name — название репозитория.
- Description — краткое текстовое описание репозитория.
- Programming languages — перечень языков программирования.
- Forks count — количество форков.
- Stars count — количество звёзд.
- Commits count — общее количество коммитов.
- Pull requests count — количество открытых pull request.
- Contributors count — количество контрибьюторов.
- Releases counts — сколько релизов опубликовано.
- Url — прямая ссылка на репозиторий.

### 2.5 Парсинг. Промежуточные выводы

- Был получен датасет (по тегам AI и ML с 10 базовыми признаками), состоящий из 1953 наблюдений, отражающих основные метрики популярности и вовлеченности сообщества при работе с открытыми репозиториями в области исскуственного интеллекта и машинного обучения, а также технологические стеки, используемые в репозиториях.
- Результат был сохранен в единый CSV-файл (github_repositories.csv) для удобства дальнейшего взаимодействия с полученными данными другими участниками команды.
- При получении данных с помощью технологии веб-скрапинга была обнаружена проблема ограниченности данного метода. Некоторые данные, например, дата создания репозитория могут быть получены только с помощью API GitHub. С целью отражения знаний полученных на курсе и дальнейшего удобства визуализации данных,
данный признак был сгенерирован исскуственно.

# 3. Очистка данных

Столбец Programming languages приведён к структурированному виду (список языков в каждой записи). Дополнительно создан флаг languages_missing для контроля записей, где языки отсутствуют.

### 3.1 Общий обзор датасета

In [ ]:
df = pd.read_csv(DATA_DIR / 'github_repositories.csv')
print("Shape:", df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include="all").T

### 3.2 Обработка пропусков и заполнение описаний

In [ ]:
for col in ["Name", "Description", "Url"]:
  if col in df.columns:
    before_replacement = df[col].isna().sum()
    df[col] = df[col].replace(r"^\s*$", np.nan, regex=True)
    after_replacement = df[col].isna().sum()

    print(f'В колонке "{col}" заменено {after_replacement - before_replacement} значений на NaN.')

Пустые значения нормализованы: в текстовых колонках пустые строки заменены на NaN, при этом структурные данные не затрагиваются.

In [ ]:
if "Description" in df.columns:
    before_fillna = df["Description"].isna().sum()
    df["Description"] = df["Description"].fillna("")
    after_fillna = df["Description"].isna().sum()

    print(f'В колонке "Description" заполнено {before_fillna} значений NaN пустыми строками.')


Description: пропуски заполнены пустой строкой "" (чтобы анализ текста не ломался).

In [ ]:
key_cols = [c for c in ["Name", "Url"] if c in df.columns]
before = len(df)
df = df.dropna(subset=key_cols)
print("Удалено строк из-за пропусков в ключевых полях:", before - len(df))

In [ ]:
print("\nПропуски по колонкам после очистки:\n", df.isna().sum().sort_values(ascending=False))

### 3.3 Удаление дубликатов

In [ ]:
if "Url" in df.columns:
    df = df.drop_duplicates(subset=["Url"], keep="first")

df = df.drop_duplicates()

print("Удалено дубликатов:", before - len(df))

Удалили строки-дубликаты (сначала по Url, затем полностью совпадающие), чтобы каждый репозиторий учитывался один раз и метрики не дублировались.

### 3.5 Преобразование столбца Programming languages и флага languages_missing

In [ ]:
def parse_languages(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]

    s = str(x).strip()
    if s in ("", "[]", "nan", "None"):
        return []

    if s.startswith("[") and s.endswith("]"):
        inner = s[1:-1].strip()
        if inner == "":
            return []
        parts = inner.split(",")
        langs = []
        for p in parts:
            p = p.strip()
            if (p.startswith("'") and p.endswith("'")) or (p.startswith('\"') and p.endswith('\"')):
                p = p[1:-1]
            p = p.strip()
            if p:
                langs.append(p)
        return langs

    return [s] if s else []

# Применяем функцию и подсчитываем, сколько строк изменилось
changes = 0
for idx, value in df["Programming languages"].items():
    original_value = df.at[idx, "Programming languages"]
    transformed_value = parse_languages(original_value)

    if original_value != transformed_value:
        changes += 1

    df.at[idx, "Programming languages"] = transformed_value

print(f"Всего изменилось значений: {changes}")

### 3.5 Очистка от выбросов по нескольким метрикам

In [ ]:
k = 1.5
need_at_least = 3  # удаляем, если выбросов >= 3 метрик

releases_col = "Releases count" if "Releases count" in df.columns else "Releases counts"
count_cols = [c for c in ["Forks count","Stars count","Commits count","Pull requests count","Contributors count",releases_col] if c in df.columns]

flags = pd.DataFrame(index=df.index)

for col in count_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    right = Q3 + k * IQR
    flags[col] = df[col] > right

to_drop = flags.sum(axis=1) >= need_at_least

before = len(df)
df = df[~to_drop].copy()
print("Удалено строк (выбросы по >=3 метрикам):", before - len(df))

In [ ]:
df.shape

Для выбросов использовали множественный критерий: строка удалялась только если превышала порог IQR (Q3+1.5⋅IQR) сразу по нескольким (≥3) метрикам. Такой подход выбран как более мягкий, поскольку GitHub-показатели имеют выраженный “длинный хвост”, и одиночные экстремальные значения часто соответствуют реально популярным репозиториям, поэтому множественный критерий снижает риск удаления валидных наблюдений.

### 3.6 Очистка. Промежуточные выводы

- **Пропуски (3.2):** пустые строки нормализованы в `Name/Description/Url` (0 значений заменено на `NaN`), `Description` заполнена без потерь (0 заполнений), ключевые поля без пропусков (`0` строк удалено), итоговые пропуски в столбцах — 0.
- **Дубликаты (3.3):** удалений не потребовалось (`0` записей), структура датасета сохранена.
- **Нормализация языков (3.5):** список `Programming languages` очищен и флаг `languages_missing` обновлён (изменено 20 значений).
- **Выбросы (3.5):** удалена 1 запись, превышающая порог по ≥3 метрикам; размер датасета снизился с 1953 до 1952 строк.


# 4. Создание новых признаков

### 4.1 Генерация временных меток Created и Last updated

Как упомянуто было ранее, при парсинге мы встретились с проблемой невозможности спарсить дату создания репозитория через веб-скрэйпинг, значит сгенерируем её синтетически.

Created - дата создание репозитория  
Last updated - время последнего изменения  
Допустим, что репозитории созданы в период 2018–2024

In [ ]:
np.random.seed(42)

n = len(df)

start_date = pd.Timestamp("2018-01-01")
end_date   = pd.Timestamp("2024-12-31")

created_random = start_date + pd.to_timedelta(np.random.randint(0, (end_date - start_date).days + 1, size=n),unit="D")
df["Created"] = created_random

delta_days = np.random.randint(0, 365 * 3, size=n)
df["Last updated"] = df["Created"] + pd.to_timedelta(delta_days, unit="D")
df["Last updated"] = df["Last updated"].clip(upper=end_date)
df.head(2)

### 4.2 Подробное описание исходных столбцов

- **Name** - Название репозитория на GitHub. Это короткое имя, которое отображается в URL и в интерфейсе GitHub.  
- **Description** - Текстовое описание репозитория, которое автор указал в настройках. В неи обычно пишут, что делает проект, для кого он, какие у него основные фичи.  
- **Programming languages** - Список языков, которые GitHub определил в этом репозитории по коду.  
- **Forks count** - Количество форков репозитория. Форк — это копия проекта, которую кто-то сделал к себе в аккаунт, чтобы доработать/изменять код. Чем больше форков, тем чаще проект используют как основу для своих решений, фреймворков, экспериментов.  
- **Stars count** - Количество «звёзд» — лайков, которые пользователи поставили репозиторию на GitHub. Это простой индикатор популярности / интереса к проекту.  
- **Commits count** - Общее количество коммитов (фиксаций изменений) в репозитории. Это показывает, сколько раз код изменяли за всё время существования проекта.  
- **Pull requests count** - Общее количество pull requests (PR) в репозитории. PR — это запросы на вливание изменений, часто от внешних участников. Это важный индикатор того, насколько активно комьюнити участвует в развитии проекта: много PR → много внешних вкладчиков/изменений.  
- **Contributors count** - Количество контрибьюторов — людей, которые внесли изменения в код (хотя бы один коммит). Это размер команды/сообщества разработчиков. Репозиторий с 2 контрибьюторами и репозиторий с 300 контрибьюторами — очень разные по масштабу проекты.  
- **Releases count** - Сколько релизов (официальных версий) опубликовано в репозитории через GitHub Releases. Это говорит о том, насколько «продуктово» ведётся проект: есть ли формальные версии, changelog, релизные циклы. У библиотек/фреймворков обычно релизов много, у демо/песочниц — мало или вообще нет.  
- **Url** - Прямая ссылка на репозиторий на GitHub.

### 4.3 Расчёт новых метрик вовлечённости и стека

- **Stars per contributor** - отражает насколько много звёзд приходится на одного участника, позволяет сравнить маленькие и большие команды.  

Гипотеза: у маленьких, но популярных библиотек значение может быть очень высоким, а у огромных инфраструктурных проектов — ниже, при том что звёзд много. Попробовать сравнить медианы Stars per contributor для разных типов проектов: библиотека/платформа, Python vs. не-Python и т.д.

In [ ]:
df["Stars per contributor"] = round(df["Stars count"] / (df["Contributors count"] + 1), 1)
df.head(1)

- **Commits per contributor** - интенсивность вклада одного участника: среднее число коммитов на одного контрибьютора.
Можно интерпретировать как «насколько активно работает каждый участник».

Гипотеза: у популярных репозиториев выше Commits per contributor, чем у непопулярных, посмотреть, отличается ли это между «демо»-репозиториями и серьёзными фреймворками.

In [ ]:
df["Commits per contributor"] = round(df["Commits count"] / (df["Contributors count"] + 1), 1)
df.head(1)

- **Community openness** - доля изменений через PR (индекс открытости). Если Community openness близко к 0, значит почти всё делают прямыми коммитами, команда закрытая, а если ближе к 1, это значит что основная жизнь идёт через PR, проект открыт к внешним вкладчикам.

Гипотеза: популярные проекты (по звёздам) чаще имеют высокий Community openness, а образовательные демо-репозитории, наоборот, почти не принимают PR.

In [ ]:
df["Total changes"] = df["Commits count"] + df["Pull requests count"]
df["Community openness"] = df["Pull requests count"] / (df["Total changes"] + 1)
df.head(1)

- **Num languages** - стек технологий и «сложность» проекта, то есть, чем больше языков, тем более «широкий» стек (frontend+backend+infra и т.д.).<br>

Гипотеза: проекты с большим числом использованных языков, особенно когда это включает такие популярные языки, как Python и TypeScript, имеют более широкий стек технологий, что может указывать на большую гибкость и сложность проекта. Большая часть топовых репозиториев, как показывает статистика, часто использует такие языки, как Python для backend-разработки и TypeScript для frontend.

In [ ]:
df["Num languages"] = df["Programming languages"].apply(len)
df.head(1)

In [ ]:
backend_set  = {"Python", "Java", "Go", "C++", "Rust", "C#", "Ruby"}
frontend_set = {"JavaScript", "TypeScript", "HTML", "CSS"}
infra_set    = {"Dockerfile", "Shell"}
data_set     = {"SQL", "PLpgSQL", "R"}

def classify_stack(langs):
    langs = set(langs)
    has_backend  = len(langs & backend_set) > 0
    has_frontend = len(langs & frontend_set) > 0
    has_infra    = len(langs & infra_set) > 0
    has_data     = len(langs & data_set) > 0
    return pd.Series({
        "Has backend": has_backend,
        "Has frontend": has_frontend,
        "Has infra": has_infra,
        "Has data": has_data,
        "Stack breadth": sum([has_backend, has_frontend, has_infra, has_data]),
    })

stack_features = df["Programming languages"].apply(classify_stack)
df = pd.concat([df, stack_features], axis=1)

df.head(1)

- **Hype score** - Hype-индекс» по модным словам (LLM, agents, RAG…) отражает, чем занимается проект.

Гипотеза: проекты, содержащие больше таких «модных» слов, как LLM, agents, RAG, чаще всего являются новыми, актуальными и активно развивающимися, что может указывать на их высокую востребованность и современность. Гипотеза основывается на том, что проекты, ориентированные на новейшие технологии, привлекают больше внимания, пользователей и контрибьюторов.

In [ ]:
buzzwords = ["llm", "agent", "rag", "workflow", "chatbot", "no-code", "low-code"]

def hype_score(s):
    return sum(word in s for word in buzzwords)

df["Hype score"] = df["Description"].apply(hype_score)
df.head(1)

In [ ]:
df.shape

### 4.4 Блок разведочного анализа по активности сообществ
Здесь фиксируем цель всех следующих шагов: понять, какие метрики (звезды, форки, коммиты) сильнее всего характеризуют крупные open-source проекты и как они взаимосвязаны между собой.


#### Словарь новых признаков после обогащения:

- **Created** — сгенерированная дата создания репозитория, нужна для условного временного анализа.
- **Last updated** — дата последнего изменения (обрезана по 2024-12-31), позволяет оценить свежесть проектов.
- **Stars per contributor** — среднее число звёзд на участника, сравнивает популярность с учётом размера команды.
- **Commits per contributor** — среднее число коммитов на участника, отражает типичную нагрузку на одного разработчика.
- **Total changes** — сумма коммитов и PR, базовая метрика общего объёма изменений.
- **Community openness** — доля изменений через pull request, показывает, насколько проект открыт к внешнему вкладу.
- **Num languages** — количество языков в стеке, грубая оценка технологической широты.
- **Has backend / Has frontend / Has infra / Has data** — булевы флаги наличия соответствующих компонентов в стеке.
- **Stack breadth** — суммарное число найденных компонент стека, быстрый индикатор полноты (backend+frontend+infra+data).
- **Hype score** — счётчик модных слов из описания (LLM, agents, RAG и т.д.), помогает выделить трендовые темы.


### 4.6 Обогащение. Промежуточные выводы

* Добавили временные признаки (created/updated) и сгруппировали активность по дням, чтобы сравнивать репозитории по темпу роста.
* Посчитали итоговые метрики вовлечённости (отношения stars/forks к времени жизни, PR/issue-рейты, доля контрибьюторов), чтобы видеть, насколько проект живой.
* Описали словарь новых признаков и флаги по стеку (например, наличие Python/JS/Java), чтобы упростить фильтрацию и сравнение языков.
* После обогащения финальный датасет вырос до 23 столбцов и позволяет строить более содержательные графики по активности сообществ.


# 5. Анализ данных и визуализация

## 5.1 Описательная статистика

#### Исследуем базовую статистику по ключевым числовым признакам, чтобы оценить порядок величин и разброс активности в выборке.


In [ ]:
df_without_cols = df.drop(['Created', 'Last updated'], axis=1) # исключаем признаки типа "дата"
df_without_cols.describe().round(2).T

**Выводы по описательной статистике.** Распределения звёзд и форков сильно скошены вправо: у большинства проектов десятки/сотни оценок, но есть небольшой слой сверхпопулярных репозиториев. PR и релизы часто нулевые, поэтому активность оформлена не у всех проектов. Число контрибьюторов колеблется от одиночных разработчиков до крупных команд, что говорит о разном уровне зрелости проектов.



#### Построим матрицу корреляций по основным числовым признакам популярности и активности репозиториев.


In [ ]:
corr_cols = [
    'Stars count', 'Forks count', 'Commits count', 'Pull requests count',
    'Contributors count', 'Stars per contributor', 'Commits per contributor',
    'Community openness', 'Num languages'
]
corr_df = df[corr_cols].corr().round(2)
corr_df

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Корреляция ключевых числовых метрик')
plt.show()

**Вывод по корреляции.** Самая сильная связь наблюдается между числами звёзд и форков, чуть слабее — между звёздами и количеством контрибьюторов. Почти отсутствует линейная связь между числом языков и активностными метриками, что видно по низким корреляциям в соответствующих строках/столбцах.

## 5.2 Популярность языков и стэков

#### Посмотрим, какие языки чаще всего встречаются как основной язык репозитория.


In [ ]:

lang_series = df['Programming languages'].apply(
    lambda x: x if isinstance(x, list) else ast.literal_eval(x) if isinstance(x, str) else []
)
lang_counts = lang_series.explode().value_counts()
language_percent = (lang_counts / lang_counts.sum()) * 100

# Выделяем языки с долей >= 3%, остальные объединяем в "Other"
top_languages = language_percent[language_percent >= 3]
other_percent = 100 - top_languages.sum()
languages_for_pie = pd.concat([top_languages, pd.Series({'Other': other_percent})])

plt.figure(figsize=(8, 8))
colors = sns.color_palette('Set3', n_colors=len(languages_for_pie))
languages_for_pie.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=colors, legend=False)
plt.title('Распределение языков в проектах')
plt.ylabel('')
plt.tight_layout()
plt.show()


### Средние звёзды по стэку

**Цель подсчёта средних звёзд по стэку.** Хотим понять, дают ли разные комбинации языков заметное преимущество по популярности и выделяются ли гибридные стэки среди лидеров.


In [ ]:

stacked_langs = lang_series.apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))
mean_stars_by_stack = (
    df.assign(Stack=stacked_langs)
      .groupby('Stack')['Stars count']
      .mean()
      .sort_values(ascending=False)
      .head(10)
)
mean_stars_by_stack


**Выводы по языкам и популярности.** Python остаётся главным языком выборки; далее идут JavaScript/TypeScript и Shell. В топовых по звёздам стэках чаще встречается сочетание Python с фронтенд-языками, то есть гибридные проекты собирают больше внимания, чем одиночные инфраструктурные репозитории.


### План визуализаций по языкам
Мы исследуем популярность технологий и связь метрик активности с популярностью репозиториев: топовые языки, распределения звёзд/форков по стеку, форма зависимости Stars-Forks, корреляции активности (Commits, PRs, Contributors, Stars) и влияние количества контрибьюторов/PR на звёзды.
Отдельно смотрим на `hype score`, чтобы сопоставить модные темы с популярностью.


### Топ-10 языков
Визуализируем топ-10 основных языков, чтобы оценить технологическую популярность в выборке.


In [ ]:

# Преобразуем столбец 'Created' в datetime и разворачиваем списки языков
created_dt = pd.to_datetime(df['Created'], errors='coerce')
timeline = pd.DataFrame({
    'Created': created_dt,
    'Programming languages': lang_series
}).explode('Programming languages').dropna(subset=['Programming languages'])

# Отбираем топ-10 языков по количеству репозиториев
top_10_languages = timeline['Programming languages'].value_counts().head(10).index
trend_data = (
    timeline[timeline['Programming languages'].isin(top_10_languages)]
    .assign(**{'Year-Month': lambda d: d['Created'].dt.to_period('M').dt.to_timestamp()})
    .groupby(['Year-Month', 'Programming languages'])
    .size()
    .reset_index(name='Repositories')
)

plt.figure(figsize=(14, 8))
sns.lineplot(
    data=trend_data,
    x='Year-Month',
    y='Repositories',
    hue='Programming languages',
    marker='o',
    dashes=False,
    palette='tab10'
)
plt.title('Прирост создания репозиториев по топ-10 языкам', fontsize=16)
plt.xlabel('Год-Месяц', fontsize=12)
plt.ylabel('Количество репозиториев', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.legend(title='Языки', loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()



**Вывод по топ-10 языкам.** Python и JavaScript уверенно лидируют по числу репозиториев; TypeScript, Shell и Jupyter Notebook формируют следующий кластер, отражая популярность веб- и data-стеков. Лидеры сохраняют отрыв на всём горизонте наблюдения.


### Распределения звёзд/форков по топ-5 языкам
Сравниваем разброс популярности по стеку через boxplot/violin.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.violinplot(
    data=lang_long,
    x='Stars count',
    y='Programming languages',
    cut=0,
    ax=axes[0]
)
axes[0].set_title('Распределение звёзд по языкам')
axes[0].set_xlabel('Количество звёзд')
axes[0].set_ylabel('Язык')
axes[0].set_xlim(0, stars_q99)

sns.boxplot(
    data=lang_long,
    x='Forks count',
    y='Programming languages',
    ax=axes[1]
)
axes[1].set_title('Распределение форков по языкам')
axes[1].set_xlabel('Количество форков')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

**Вывод по распределениям.** У Python самый широкий разброс звёзд и форков: встречаются как небольшие, так и очень популярные проекты. JavaScript и TypeScript сгруппированы ближе к средним значениям, а Shell и Jupyter Notebook дают более компактные распределения, что отражает их нишевые сценарии.


## 5.3 Связи метрик и активность

#### Scatter Stars–Forks в лог-шкале
Проверяем форму связи между звёздами и форками в логарифмическом масштабе.


In [ ]:
scatter_df = df[(df['Stars count'] > 0) & (df['Forks count'] > 0)]
low = scatter_df[['Stars count', 'Forks count']].quantile(0.01)
high = scatter_df[['Stars count', 'Forks count']].quantile(0.99)
scatter_df = scatter_df[
    scatter_df['Stars count'].between(low['Stars count'], high['Stars count']) &
    scatter_df['Forks count'].between(low['Forks count'], high['Forks count'])
]

plt.figure(figsize=(7, 6))
hb = plt.hexbin(
    scatter_df['Forks count'],
    scatter_df['Stars count'],
    gridsize=40,
    cmap='magma',
    norm=matplotlib.colors.LogNorm(),
    mincnt=1
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Forks count (log)')
plt.ylabel('Stars count (log)')
plt.title('Зависимость Stars vs. Forks\nлог-шкала и плотность точек')
cb = plt.colorbar(hb)
cb.set_label('log(количество репозиториев в бине)')
plt.tight_layout()


**Вывод по Stars–Forks.** В логарифмической шкале точки выстраиваются почти в линию: чем больше форков, тем больше звёзд. Сверхпопулярные репозитории формируют редкие выбросы, а основное облако показывает типичную степенную зависимость.


#### Коррелограмма ключевых метрик
Ищем взаимосвязи между Stars, Forks, Commits, PR и Contributors.


In [ ]:
activity_cols = ['Stars count', 'Forks count', 'Commits count', 'Pull requests count', 'Contributors count']
sns.set_theme(style='whitegrid')
g = sns.pairplot(
    df[activity_cols],
    corner=True,
    diag_kind='hist',
    plot_kws={'alpha': 0.3, 's': 20}
)
g.fig.suptitle('Связи метрик вовлечённости и популярности', y=1.02)
plt.show()


**Вывод по коррелограмме.** Самая заметная связь — между звёздами и форками. Больше коммитов и PR идёт рука об руку с ростом числа контрибьюторов, тогда как количество языков почти не влияет на популярность.


#### Трендлайны влияния контрибьюторов и PR на звёзды
Сравниваем наклоны линий зависимости Stars от числа контрибьюторов и PR.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
for ax, (xcol, title) in zip(axes, [('Contributors count', 'влияние контрибьюторов'), ('Pull requests count', 'влияние PR')]):
    reg_df = df[(df['Stars count'] > 0) & (df[xcol] > 0)]
    reg_df = reg_df[
        (reg_df['Stars count'] < reg_df['Stars count'].quantile(0.995)) &
        (reg_df[xcol] < reg_df[xcol].quantile(0.995))
    ]
    sns.regplot(
        data=reg_df,
        x=xcol,
        y='Stars count',
        scatter_kws={'alpha': 0.25, 's': 18},
        line_kws={'color': 'darkred'},
        ci=None,
        ax=ax
    )
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(f"Trendline: Stars ~ {title}\nдля количественной оценки влияния")
    ax.set_xlabel(f"{xcol} (log)")
    ax.set_ylabel('Stars count (log)')
plt.tight_layout()


**Вывод по трендовым линиям.** При увеличении числа контрибьюторов звёзды растут быстрее, чем при росте количества PR: дополнительный участник даёт больший вклад, а объём PR работает скорее как поддерживающий фактор.


## 5.4 Итоговые выводы EDA
- **Главные языки.** Большая часть репозиториев построена на Python; далее идут JavaScript/TypeScript и другие вспомогательные языки.
- **Распределения популярности.** У звёзд и форков длинные правые хвосты: большинство проектов набирают умеренные показатели, но есть редкие лидеры с десятками тысяч оценок.
- **Связи метрик.** Звёзды растут вместе с форками; активность (коммиты, PR) и число контрибьюторов движутся в одну сторону, но эффект слабее, чем у пары Stars–Forks.
- **Роль сообщества.** Больше участников даёт более заметный прирост звёзд, чем просто увеличение числа PR, поэтому вовлечение людей важнее, чем объём изменений.


# 6. Итоговые выводы исследования (общий шаблон)

>Ребята, замените подсказки своими итогами.

**Предлагаемый формат заполнения:**
- Матвей: кратко о составе и сборе датасета (объём, источники, качество парсинга).
- Дина: ключевые выводы об очистке и качестве данных (пропуски, дубликаты, выбросы).
- Лена: влияние созданных признаков и как они улучшают анализ активности/стека.
- Дима: основные инсайты EDA и визуализаций о популярности и взаимосвязях метрик.
- Итог: 3–4 буллета с общими выводами и 1 пункт с ограничениями/идеями для будущей работы.

